In [1]:
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
# change URL base on the category
BASE_URL = 'https://www.betclic.fr/'
URL = 'https://www.betclic.fr/tennis-s2'

def get_match_links(url: str=URL) -> dict:

    response = requests.get(url)
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.text, 'html.parser')
        href_attributes = [BASE_URL+a.get('href') for a in soup.find_all('a') if '/tennis-s2/' in a.get('href')]
        matchs_link = [href.encode('ascii', 'ignore').decode('unicode_escape') for href in href_attributes]

        return {"Error": False, "links": matchs_link}
    else:
        return{"Error": True, "Message": f"Unable to retrieve the webpage. Status code: {response.status_code}"}  

def create_tennis_driver(link: str):

    # set up Chrome driver
    options = Options()
    #options.add_argument('--no-sandbox')
    #options.add_argument('--headless')
    #options.add_argument('--disable-gpu')
    #options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Navigate to the webpage you want to scrape
    driver.get(link)

    try:
        # coockies
        driver.find_element("xpath", '//*[@id="popin_tc_privacy_button_2"]').click()
        driver.implicitly_wait(1)
        # click to "points et service"
        x_path = '//*[@id="matchHeader"]/div/sports-category-filters/bcdk-tabs/div/div/div/div[last()]/span'
        driver.find_element("xpath", x_path).click()
        driver.implicitly_wait(2)
        return driver
    
    except Exception as e:
        return None  

def retrieve_tennis_point_service(driver: webdriver):
    class_name = 'block'
    point_service = driver.find_element(By.CLASS_NAME, class_name)
    info = point_service.text
    return info

In [4]:
if __name__ == '__main__':
    
    match_links = get_match_links()

    if match_links.get('Error'):
        print("we have a problem of too many requests")
    else:
        links = match_links.get('links')

    all_text = []

    for url in links[:5]:
        driver = create_tennis_driver(url)
        if driver is None:
            continue
        else:
            info = retrieve_tennis_point_service(driver)
            try:
                result = re.search(r'Aces(.*?)0 selection', info, re.DOTALL)
                if result:
                    extracted_text = result.group(1).strip()
                    print(extracted_text)
                all_text.append(extracted_text)
            except Exception:
                print("no Aces found")
    text = '|'.join(elm for elm in all_text)

    # Writing to a file
    with open("tennis_data.txt", "w") as file:
        file.write(text)


no Aces found
Points 
Vainqueur Point 4 (Set 2, Jeu 12)
A. Mannarino
2,25
M. Giron
1,35
Aces
Plus grand nombre d'aces
A. Mannarino
7,00
Nul
5,00
M. Giron
1,10
Nombre total d'aces dans le match
+ de 12,5
1,70
- de 12,5
1,70
Nombre d'aces inscrits par le joueur 1 dans le match
+ de 5.5
1,50
- de 5.5
1,90
Nombre d'aces inscrits par le joueur 2 dans le match
+ de 7.5
1,70
- de 7.5
1,70
Jeux de service
Aces
Vainqueur du match et Nombre total d'aces
Taylor Fritz et + de 14,5
2,10
Taylor Fritz et - de 14,5
2,10
Nuno Borges et + de 14,5
7,00
Nuno Borges et - de 14.5
7,00
Premier ace
Taylor Fritz
1,18
Nuno Borges
3,25
Plus grand nombre d'aces
Taylor Fritz
1,01
Nul
8,00
Nuno Borges
8,00
Nombre total d'aces dans le match
+ de 14.5
1,70
- de 14.5
1,70
Nombre d'aces inscrits par le joueur 1 dans le match
+ de 9.5
1,70
- de 9.5
1,70
Nombre d'aces inscrits par le joueur 2 dans le match
+ de 4.5
1,70
- de 4.5
1,70
Jeux de service
Score exact sur le 1er jeu de service Taylor Fritz
Taylor Fritz Vainqueu